In [ ]:
!pip install conllu
import numpy as np 
import pandas as pd
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import random


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
!pip install mtranslate
from mtranslate import translate
ads = pd.read_csv('/work/playing-with-ads/ads_en_us.csv')
ads = ads.drop_duplicates(subset = 'value')
ads = ads.rename(columns = {'value':'text'})
ads = ads.iloc[0:29500]
ads.text = ads.text.apply(lambda x: translate(x, "ru", "en"))
ads.to_csv('ads_ru.csv', index = False)

In [ ]:
from io import open
from conllu import parse_incr
lst = []
data_file = open("/work/tagged_texts/vktexts.txt", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    lst.append(tokenlist.metadata['text'])
    if len(lst) >= 10000:
        break
data_file = open("/work/tagged_texts/fbtexts.txt", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    lst.append(tokenlist.metadata['text'])
    if len(lst) >= 20000:
        break
data_file = open("/work/tagged_texts/twtexts.txt", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    lst.append(tokenlist.metadata['text'])
    if len(lst) >= 30000:
        break

In [ ]:
df = pd.read_csv('/work/contextAdvertising1.csv')
df = df.dropna()
df['text'] = df.atitle + df.atext
df = df.drop_duplicates(subset = 'text')
df['is_ad'] = 1
df = df.sample(25000, random_state = 1)

In [ ]:
twi = pd.read_csv('/work/rusentitweet_full.csv')
twi['is_ad'] = 0

In [ ]:
ads =  pd.read_csv('ads_ru.csv')
ads['is_ad'] = 1
ads = ads[['text', 'is_ad']]
ads = ads.dropna()

In [ ]:
anek = pd.read_csv('/work/vk-anekdots/anekdots_result.csv')
anek['is_ad'] = 0
anek = anek.dropna()
anek = anek.drop_duplicates()
anek = anek.sample(30000, random_state = 1)
fin = pd.read_csv('/work/telegram-financial-sentiment-ru/mt-cleaned-labeled5.csv')
fin['is_ad'] = 0
fin = fin.dropna()
fin = fin.drop_duplicates()
fin = fin.sample(67000, random_state = 1).rename(columns = {'message':'text'})
fin = fin[['text', 'is_ad']]

In [ ]:
tx = pd.DataFrame({'text': lst, 'is_ad':[0 for i in range(len(lst))]})
tx = tx.drop_duplicates()
df = pd.concat([df[['text',  'is_ad']], twi[['text',  'is_ad']], tx, anek, fin, ads])


In [ ]:
df = df.sample(frac=1, random_state = 1)

In [ ]:
# df.to_csv('text.csv', index = False)

2. Include Recurrent Layers:
Recurrent layers like LSTM or GRU can capture sequential dependencies in the text data. You can add a Bidirectional LSTM layer.

In [ ]:
#never gonna run on this machine

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 1250
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

training_padded = pad_sequences(tokenizer.texts_to_sequences(train.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(tokenizer.texts_to_sequences(valid.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padded = pad_sequences(tokenizer.texts_to_sequences(test.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)

vocab_size = len(tokenizer.word_index) + 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

# Adam = tf.keras.optimizers.Adam(learning_rate=0.00001)
Adam = 'Adam'
model.compile(loss='binary_crossentropy',optimizer = Adam, metrics=['AUC'])
model.fit(training_padded, train.is_ad, epochs=num_epochs, validation_data = (valid_padded, valid.is_ad), verbose=1) #validatio

Epoch 1/3
2024-06-24 15:36:05.836651: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 500000000 exceeds 10% of free system memory.
  63/3125 [..............................] - ETA: 4:15:00 - loss: 0.5811 - auc: 0.5663

KeyboardInterrupt: 

3. Add Dropout Layers:
Dropout layers help prevent overfitting by randomly dropping units during training.

In [ ]:
#OK, but needs more playing with

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 1250
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

training_padded = pad_sequences(tokenizer.texts_to_sequences(train.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(tokenizer.texts_to_sequences(valid.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padded = pad_sequences(tokenizer.texts_to_sequences(test.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)

vocab_size = len(tokenizer.word_index) + 1

num_epochs = 3
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

# Adam = tf.keras.optimizers.Adam(learning_rate=0.00001)
Adam = 'Adam'
model.compile(loss='binary_crossentropy',optimizer = Adam, metrics=['AUC'])
model.fit(training_padded, train.is_ad, epochs=num_epochs, validation_data = (valid_padded, valid.is_ad), verbose=1) #validatio

2024-06-24 16:00:25.269106: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-24 16:00:25.269157: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-24 16:00:25.269187: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-cd1fab30-834a-44c0-9ca0-4749a42969a7): /proc/driver/nvidia/version does not exist
2024-06-24 16:00:25.269570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Epoch 1/3
2024-06-24 16:00:25.488405: W tensorflow/core/framework/cpu_alloca

Batch Normalization:
Adding batch normalization can stabilize and accelerate the training process.

In [ ]:
# On every fucking layer? No way this won't fuck it up. Try keeping only last one

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 1250
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

training_padded = pad_sequences(tokenizer.texts_to_sequences(train.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(tokenizer.texts_to_sequences(valid.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padded = pad_sequences(tokenizer.texts_to_sequences(test.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)

vocab_size = len(tokenizer.word_index) + 1

num_epochs = 3
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(24, activation='relu'),
    # tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])


# Adam = tf.keras.optimizers.Adam(learning_rate=0.00001)
Adam = 'Adam'
model.compile(loss='binary_crossentropy',optimizer = Adam, metrics=['AUC'])
model.fit(training_padded, train.is_ad, epochs=num_epochs, validation_data = (valid_padded, valid.is_ad), verbose=1) #validatio

Epoch 1/3
2024-06-24 16:08:11.110722: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 500000000 exceeds 10% of free system memory.
3125/3125 [==============================] - 146s 46ms/step - loss: 0.0949 - auc: 0.9884 - val_loss: 0.0300 - val_auc: 0.9987
Epoch 2/3
3125/3125 [==============================] - 145s 46ms/step - loss: 0.0386 - auc: 0.9972 - val_loss: 0.4559 - val_auc: 0.9134
Epoch 3/3
3125/3125 [==============================] - 147s 47ms/step - loss: 0.0274 - auc: 0.9984 - val_loss: 0.0618 - val_auc: 0.9915


5. Optimizer Tuning:
Adjusting the learning rate of the Adam optimizer can have a significant impact on model performance.

In [ ]:
#nope
Adam = tf.keras.optimizers.Adam(learning_rate=0.00001)
model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=['AUC'])

6. Increase Epochs and Use Early Stopping:
Increasing the number of epochs and implementing early stopping can help the model learn better while preventing overfitting.

In [ ]:
# Good, but not as architecture changing option
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)

vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 8
max_length = 1250
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

training_padded = pad_sequences(tokenizer.texts_to_sequences(train.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
valid_padded = pad_sequences(tokenizer.texts_to_sequences(valid.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padded = pad_sequences(tokenizer.texts_to_sequences(test.text), maxlen=max_length, padding=padding_type, truncating=trunc_type)
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)

vocab_size = len(tokenizer.word_index) + 1

num_epochs = 3
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # tf.keras.layers.Conv1D(20, 4), 
    tf.keras.layers.GlobalAveragePooling1D(),
    # tf.keras.layers.Dense(300, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    # tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200)),
    tf.keras.layers.Dense(200, activation='relu'),
    # tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
    # tf.keras.layers.BatchNormalization(1),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

# Adam = tf.keras.optimizers.Adam(learning_rate=0.00001)
Adam = 'Adam'
model.compile(loss='binary_crossentropy',optimizer = Adam, metrics=['AUC'])
model.fit(training_padded, train.is_ad, epochs=10, validation_data=(valid_padded, valid.is_ad), callbacks=[early_stopping], verbose=1)


2024-06-24 17:25:44.059933: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-06-24 17:25:44.059979: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-06-24 17:25:44.060005: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-cd1fab30-834a-44c0-9ca0-4749a42969a7): /proc/driver/nvidia/version does not exist
2024-06-24 17:25:44.060390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Epoch 1/10
2024-06-24 17:25:44.373320: W tensorflow/core/framework/cpu_alloc

7. Use Pre-trained Embeddings:
Utilizing pre-trained embeddings like GloVe or Word2Vec can provide the model with richer semantic information from the start.

In [ ]:
#never gonna run on this machine

import numpy as np

embedding_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cd1fab30-834a-44c0-9ca0-4749a42969a7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>